<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/cs_whisper%20small_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-g98jwals
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-g98jwals
  Resolved https://github.com/huggingface/transformers to commit 35551f9a0f66a22de4971b4a51b3c172d3b87f95
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [4]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm


In [5]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "cs", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 1123
})

In [8]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [9]:
model = whisper.load_model("small")

100%|███████████████████████████████████████| 461M/461M [00:05<00:00, 94.6MiB/s]


In [10]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [11]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [13]:
text

,0
0,tato norma posiluje pozici evropské agentury p...
1,dámy a pánové útok na židy je útok na nás samo...
2,zatímco se na půdě evropských institucí dohodl...
3,paní předsedající když hovoříme o humanitární ...
4,vážené poslankyně vážení poslanci zastupuji dn...
...,...
1118,ale vzhledem ke stárnutí populace a dalším výz...
1119,pane předsedající já myslím že můj přítel rich...
1120,erasmus.
1121,do budoucna je zřejmé a varují před tím analýz...


In [14]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [15]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,tato norma posiluje pozici evropské agentury p...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000274658203125, -0.00067138671875, -0.001...",16000
1,male,dámy a pánové útok na židy je útok na nás samo...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001861572265625, 0.000213623046875, 0.0020...",16000
2,male,zatímco se na půdě evropských institucí dohodl...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001495361328125, -0.00152587890625, -0.0089...",16000
3,male,paní předsedající když hovoříme o humanitární ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00018310546875, -6.103515625e-05, 0.000762...",16000
4,male,vážené poslankyně vážení poslanci zastupuji dn...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0001220703125, -0.000152587890625, -0.00015...",16000
...,...,...,...,...,...
1118,female,ale vzhledem ke stárnutí populace a dalším výz...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00335693359375, -0.01141357421875, 0.00012...",16000
1119,male,pane předsedající já myslím že můj přítel rich...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00079345703125, -0.001068115234375, -0.001...",16000
1120,male,erasmus.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.003448486328125, -0.00347900390625, -0.004...",16000
1121,male,do budoucna je zřejmé a varují před tím analýz...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000335693359375, 0.00469970703125, 0.00384...",16000


In [16]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,tato norma posiluje pozici evropské agentury p...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000274658203125, -0.00067138671875, -0.001...",16000
1,male,dámy a pánové útok na židy je útok na nás samo...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001861572265625, 0.000213623046875, 0.0020...",16000
2,male,zatímco se na půdě evropských institucí dohodl...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001495361328125, -0.00152587890625, -0.0089...",16000
3,male,paní předsedající když hovoříme o humanitární ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00018310546875, -6.103515625e-05, 0.000762...",16000
4,male,vážené poslankyně vážení poslanci zastupuji dn...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0001220703125, -0.000152587890625, -0.00015...",16000
...,...,...,...,...,...
1116,male,já když jsem slyšel pana komisaře jak si vymez...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001617431640625, -0.00146484375, 6.1035156...",16000
1119,male,pane předsedající já myslím že můj přítel rich...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00079345703125, -0.001068115234375, -0.001...",16000
1120,male,erasmus.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.003448486328125, -0.00347900390625, -0.004...",16000
1121,male,do budoucna je zřejmé a varují před tím analýz...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000335693359375, 0.00469970703125, 0.00384...",16000


In [17]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
8,female,nejhorší situace je v somálsku súdánu mauretán...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001007080078125, -0.001922607421875, -0.00...",16000
9,female,v letecké i železniční dopravě vize nula platí...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001800537109375, 0.0030517578125, 0.0003356...",16000
10,female,jen tak potvrdíte že před dvaceti lety lidé ne...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001007080078125, 0.0009765625, -0.00057983...",16000
11,female,v této souvislosti bych očekávala že alespoň d...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000518798828125, -0.0020751953125, -0.0010...",16000
13,female,já sama pocházím z regionu severní moravy ze s...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000457763671875, 0.00201416015625, 0.00283...",16000
...,...,...,...,...,...
1106,female,díky tomu se mnoho informací dostává k adresát...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000244140625, -9.1552734375e-05, 0.0009155...",16000
1109,female,podle dostupných informací turecký premiér dal...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00506591796875, -0.008453369140625, -0.003...",16000
1114,female,veřejné konzultace jsou běžně zveřejňovány pře...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001251220703125, 0.00177001953125, -0.00122...",16000
1117,female,liší se rozsah průvodních dokumentů opět vesmě...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00396728515625, 0.003875732421875, 0.005584...",16000


In [18]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="cs")
    return result["text"]

In [19]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [20]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.25806451612903225
Processed 2 files, WER: 0.8181818181818182
Processed 3 files, WER: 0.6774193548387096
Processed 4 files, WER: 0.38636363636363635
Processed 5 files, WER: 1.0
Processed 6 files, WER: 0.5151515151515151
Processed 7 files, WER: 0.46153846153846156
Processed 8 files, WER: 0.5454545454545454
Processed 9 files, WER: 0.44680851063829785
Processed 10 files, WER: 0.25
Processed 11 files, WER: 0.3333333333333333
Processed 12 files, WER: 0.5161290322580645
Processed 13 files, WER: 0.38095238095238093
Processed 14 files, WER: 0.3333333333333333
Processed 15 files, WER: 0.375
Processed 16 files, WER: 0.36363636363636365
Processed 17 files, WER: 0.7777777777777778
Processed 18 files, WER: 0.3157894736842105
Processed 19 files, WER: 0.45098039215686275
Processed 20 files, WER: 0.6363636363636364
Processed 21 files, WER: 0.32
Processed 22 files, WER: 0.47619047619047616
Processed 23 files, WER: 0.5714285714285714
Processed 24 files, WER: 0.3333333333333333
P

In [21]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.3333333333333333
Processed 2 files, WER: 0.5
Processed 3 files, WER: 0.23076923076923078
Processed 4 files, WER: 0.35294117647058826
Processed 5 files, WER: 0.55
Processed 6 files, WER: 0.2777777777777778
Processed 7 files, WER: 0.6666666666666666
Processed 8 files, WER: 0.14285714285714285
Processed 9 files, WER: 0.42857142857142855
Processed 10 files, WER: 0.23076923076923078
Processed 11 files, WER: 0.3333333333333333
Processed 12 files, WER: 0.3333333333333333
Processed 13 files, WER: 0.4444444444444444
Processed 14 files, WER: 0.7333333333333333
Processed 15 files, WER: 0.391304347826087
Processed 16 files, WER: 0.3333333333333333
Processed 17 files, WER: 0.6538461538461539
Processed 18 files, WER: 0.43243243243243246
Processed 19 files, WER: 0.35294117647058826
Processed 20 files, WER: 0.4482758620689655
Processed 21 files, WER: 0.1
Processed 22 files, WER: 0.3684210526315789
Processed 23 files, WER: 0.19230769230769232
Processed 24 files, WER: 0.3333333